# Домашнее задание № 10. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). Используйте метрику BLEU. Найдите лучшие (как минимум 5) переводы согласно этой метрике и проверьте действительно ли они хорошие. Если все переводы нулевые, то пообучайте модель подольше.

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Сейчас она работает только с одним текстом - это не эффективно. Можно генерировать переводы сразу для нескольких текстов (батча). Главная сложность с таким подходом состоит в том, что генерируемые тексты будут заканчиваться в разное время и нужно сделать столько итераций, сколько нужно для завершения всех текстов (т.е. условие на то, что последний токен не равен [END] в текущем коде не сработает).
ВАЖНО - недостаточно просто изменить входной аргумент с text на texts и добавить еще один цикл по texts! Сама модель должна вызываться на нескольких текстах!

In [ ]:
# !apt-get install unzip

In [ ]:
!pip install --upgrade keras

In [ ]:
!pip install torch==2.0.1

In [3]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
import torch
print(keras.__version__)

3.1.1


In [ ]:
# %pip install tokenizers matplotlib scikit-learn

In [4]:
# import tensorflow as tf
import keras
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2024-04-04 14:52:18--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru’

opus.en-ru-train.ru 100%[===================>] 115.72M  85.3MB/s    in 1.4s    

2024-04-04 14:52:20 (85.3 MB/s) - ‘opus.en-ru-train.ru’ saved [121340806/121340806]

--2024-04-04 14:52:20--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en’

opus.en-ru-train.en 100%[===================>]  64.62M  79.1MB/s    in 0.8s    

2024-04-04 14:52:21 (79.1 MB/s) - ‘opus.en-ru

In [6]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
text = open('opus.en-ru-train.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru', 'w')
f.write(text)
f.close()

In [7]:
en_sents = open('opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().lower().splitlines()

Пример перевода с английского на русский

In [8]:
en_sents[-1], ru_sents[-1]

('so what are you thinking?', 'ну и что ты думаешь?')

In [9]:
tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]"])
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en )

tokenizer_ru = Tokenizer(WordPiece(), )
tokenizer_ru.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_ru.pre_tokenizer = Whitespace()

trainer_ru = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]", "[START]", "[END]", ])
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru )

In [10]:
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_ru.decoder = decoders.WordPiece()

In [11]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
# tokenizer_en.save('tokenizer_en')
# tokenizer_ru.save('tokenizer_ru')

In [12]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

In [13]:
def encode(text, tokenizer, target=False):
    if target:
        return [tokenizer.token_to_id('[START]')] + tokenizer.encode(text).ids + \
                [tokenizer.token_to_id('[END]')]
    else:
        return tokenizer.encode(text).ids


In [14]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, True) for t in ru_sents]

In [15]:
# обратите внимание, что в seq2seq длины могут быть разными
max_len_en, max_len_ru = 45, 48

In [16]:
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

1

In [17]:
X_en = keras.preprocessing.sequence.pad_sequences(
              X_en, maxlen=max_len_en, padding='post', value=PAD_IDX)

X_ru_out = keras.preprocessing.sequence.pad_sequences(
              [x[1:] for x in X_ru], maxlen=max_len_ru-1, padding='post',
              value=PAD_IDX)

X_ru_dec = keras.preprocessing.sequence.pad_sequences(
              [x[:-1] for x in X_ru], maxlen=max_len_ru-1,
              padding='post', value=PAD_IDX)

In [18]:
en_sents[0]

"yeah, that's not exactly..."

In [19]:
X_en[0]

array([3280,   13, 2763,    8,   58, 2808, 5148, 2856,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1], dtype=int32)

In [20]:
ru_sents[0]

'да, но не совсем...'

In [21]:
X_ru_dec[0]

array([   2, 2588,   15, 2589, 2513, 5362, 2643,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1], dtype=int32)

In [22]:
X_ru_out[0]

array([2588,   15, 2589, 2513, 5362, 2643,    3,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1], dtype=int32)

In [23]:
# миллион примеров
X_en.shape, X_ru_out.shape

((1000000, 45), (1000000, 47))

In [24]:
(X_en_train, X_en_valid,
X_ru_dec_train, X_ru_dec_valid,
X_ru_out_train, X_ru_out_valid) = train_test_split(X_en,
                                                  X_ru_dec,
                                                  X_ru_out,
                                                  test_size=0.05)

In [25]:
def scaled_dot_product_attention(query, key, value, mask):
    # Считаем скалярное произведение между запросом (query) и ключом (key), транспонируя ключ
    matmul_qk = keras.ops.matmul(query, keras.ops.transpose(key, axes=[0, 1, 3, 2]))

    # Получаем глубину (размерность) ключа и преобразуем ее во float
    depth = keras.ops.cast(key.shape[-1], torch.float32)

    # Делим результат скалярного произведения на квадратный корень из глубины
    # Это делается для уменьшения влияния больших значений и стабилизации градиентов во время обучения
    logits = matmul_qk / keras.ops.sqrt(depth)

    # Если есть маска, применяем ее к логитам, чтобы обнулить нежелательные значения
    if mask is not None:
        logits += (mask * -1e9)

    # Применяем функцию softmax для получения весов внимания
    attention_weights = keras.ops.softmax(logits, axis=-1)

    # Умножаем веса внимания на значения (value) для получения итогового результата
    output = keras.ops.matmul(attention_weights, value)

    return output

In [26]:
class MultiHeadAttention(keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads  # количество голов для внимания
        self.d_model = d_model  # размерность вектора модели

        # Убеждаемся, что размерность модели делится нацело на количество голов
        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads  # размерность каждой головы

        # Создаем полносвязные слои для запроса, ключа и значения
        self.query_dense = keras.layers.Dense(units=d_model)
        self.key_dense = keras.layers.Dense(units=d_model)
        self.value_dense = keras.layers.Dense(units=d_model)

        # Создаем последний полносвязный слой
        self.dense = keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        # Разделяем входные данные на головы
        inputs = keras.ops.reshape(
            inputs, newshape=(batch_size, -1, self.num_heads, self.depth))
        return keras.ops.transpose(inputs, axes=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs.get('mask', None)
        batch_size = query.shape[0]

        # Пропускаем запрос, ключ и значение через соответствующие полносвязные слои
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # Разделяем запрос, ключ и значение на головы
        # то есть просто разрезаем вектора на num_heads частей
        # и сравниваем все части между собой
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Выполняем механизм внимания с масштабированным скалярным произведением
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = keras.ops.transpose(scaled_attention, axes=[0, 2, 1, 3])

        # Объединяем головы вместе (склеиваем векторы в один)
        concat_attention = keras.ops.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # Пропускаем объединенное внимание через дополнительный полносвязный слой
        # Он просто добавляет сложности нашей модели
        outputs = self.dense(concat_attention)

        return outputs


In [27]:
mha = MultiHeadAttention(d_model=10, num_heads=2)

In [28]:
inp = np.random.normal(size=(1, 2, 3))

In [29]:
inp

array([[[ 0.97277629, -0.10061499,  1.28651324],
        [-0.25863553,  0.30786601,  0.94169787]]])

In [30]:
inputs = {'query': inp,'key': inp, 'value': inp,}

In [31]:
result = mha(inputs)

In [32]:
result.shape

torch.Size([1, 2, 10])

In [33]:
# преобразуем эмбединг в q,k,v вектора
q = mha.query_dense(inputs['query'])
k = mha.query_dense(inputs['key'])
v = mha.query_dense(inputs['value'])

In [34]:
# размерность не изменилась так как мы еще не порезали на головы
q.shape

torch.Size([1, 2, 10])

In [35]:
# разрезаем каждый вектор на куски (головы)
qh = mha.split_heads(q, q.shape[0])
kh = mha.split_heads(k, k.shape[0])
vh = mha.split_heads(v, v.shape[0])

In [36]:
# мы указали num_heads 2 - поэтому размерность каждого куска 5 (10 разделить на два)
qh.shape

torch.Size([1, 2, 2, 5])

In [37]:
# keras.ops.transpose(kh).shape

In [38]:
matmul_qk = keras.ops.matmul(qh, keras.ops.transpose(kh, axes=[0, 1, 3, 2]))

In [39]:
# На выходе получится тензор вот такой размерности
# 5 изменилось на 2
# так как для каждого куска у нас по две близости (до каждого слова включая себя)
matmul_qk.shape

torch.Size([1, 2, 2, 2])

In [40]:
keras.ops.softmax(matmul_qk, axis=-1).detach().cpu().numpy().round(2)

array([[[[0.62, 0.38],
         [0.38, 0.62]],

        [[0.84, 0.16],
         [0.5 , 0.5 ]]]], dtype=float32)

In [41]:
def create_padding_mask(x):
    mask = keras.ops.cast(keras.ops.equal(x, PAD_IDX), torch.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, None, None, :]

In [42]:
create_padding_mask([[1,2,3]]) # 1 подставилась для индекса который равен PAD_IDX

tensor([[[[1., 0., 0.]]]], device='cuda:0')

In [43]:
def create_look_ahead_mask(x):
    # эта функция немножко сложная, но суть у нее достаточно простая
    # нужно создать треугольную маску, с помощью которой мы закроем
    # для каждого токена все последующие токены
    seq_len = x.shape[1]
    ones_mask = keras.ops.ones((1, seq_len, seq_len), dtype="int32")
    row_index = keras.ops.cumsum(ones_mask, axis=-2)
    col_index = keras.ops.cumsum(ones_mask, axis=-1)
    look_ahead_mask = ~ keras.ops.greater_equal(row_index, col_index)
    padding_mask = create_padding_mask(x)
    return keras.ops.maximum(look_ahead_mask, padding_mask)

In [44]:
# для декодера нам нужно замаскировать следующие токены
# так как мы пытаемся их сгенерировать
# для этого создается вот такая маска
ones_mask = keras.ops.ones((1, 3, 3), dtype="int32")
row_index = keras.ops.cumsum(ones_mask, axis=-2)
col_index = keras.ops.cumsum(ones_mask, axis=-1)
mask = ~ keras.ops.greater_equal(row_index, col_index)
mask

tensor([[[False,  True,  True],
         [False, False,  True],
         [False, False, False]]], device='cuda:0')

In [45]:
# в слое с вниманием эта маска будет использоваться чтобы занулить близости с токенами из будущего
(mask * -100)

tensor([[[   0, -100, -100],
         [   0,    0, -100],
         [   0,    0,    0]]], device='cuda:0')

In [46]:
# эта функция также проверяет и паддинг и если он есть то и его тоже замаскирует
# 1 - это паддинг айди
create_look_ahead_mask(torch.LongTensor([[1,3,2]]))

tensor([[[[1., 1., 1.],
          [1., 0., 1.],
          [1., 0., 0.]]]], device='cuda:0')

In [47]:
create_look_ahead_mask(torch.Tensor([[2,4,3]]))

tensor([[[[0., 1., 1.],
          [0., 0., 1.],
          [0., 0., 0.]]]], device='cuda:0')

In [48]:
create_look_ahead_mask(torch.Tensor([[2,4,3]]))

tensor([[[[0., 1., 1.],
          [0., 0., 1.],
          [0., 0., 0.]]]], device='cuda:0')

In [49]:
class PositionalEncoding(keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / keras.ops.power(10000, (2 * (i // 2)) / d_model)
        return keras.ops.multiply(position, angles)

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=torch.arange(0, position, dtype=torch.float32)[:, None],
            i=torch.arange(0, d_model, dtype=torch.float32)[None, :],
            d_model=d_model)
        sines = keras.ops.sin(angle_rads[:, 0::2])
        cosines = keras.ops.cos(angle_rads[:, 1::2])

        pos_encoding = keras.ops.concatenate([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[None, ...]
        return keras.ops.cast(pos_encoding, torch.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :inputs.shape[1], :]

In [50]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = keras.Input(shape=(1, 1, None), name="padding_mask")

    #call_mha
    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = keras.layers.Dropout(rate=dropout)(attention)
    attention = keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = keras.layers.Dense(units=d_model)(outputs)
    outputs = keras.layers.Dropout(rate=dropout)(outputs)
    outputs = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [51]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = keras.Input(shape=(max_len,), name="inputs")
    padding_mask = keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= keras.ops.sqrt(keras.ops.cast(d_model, torch.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    #inputs (они тут называются outputs но это просто такой нейминг,
    # этот параметр передается в encoder_layer первым и encoder_layer будет считать его inputs)
    # outputs он тут называется для удобства, так как он будет перезаписываться
    # чтобы на вход следующему блоку подавать уже не эмбединги,
    # а то что получится как результат предыдущего блока
    outputs = keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [52]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = keras.layers.Dense(units=d_model)(outputs)
    outputs = keras.layers.Dropout(rate=dropout)(outputs)
    outputs = keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [53]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = keras.Input(shape=(max_len,), name='inputs')
    enc_outputs = keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= keras.ops.sqrt(keras.ops.cast(d_model, torch.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [54]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = keras.Input(shape=(max_len[0],), name="inputs")
    dec_inputs = keras.Input(shape=(max_len[1]-1,), name="dec_inputs")

    enc_padding_mask = keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

    look_ahead_mask = keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

    dec_padding_mask = keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
                          vocab_size=vocab_size[0],
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                          max_len=max_len[0],
                        )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
                          vocab_size=vocab_size[1],
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                          max_len=max_len[1]-1,
                        )(inputs=[dec_inputs, enc_outputs,
                                  look_ahead_mask, dec_padding_mask])

    outputs = keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [55]:

L  = keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = keras.ops.cast(keras.ops.not_equal(y_true, PAD_IDX), torch.float32)
    loss = keras.ops.multiply(loss, mask)

    return keras.ops.mean(loss)

In [56]:
tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()

(30000, 30000)

In [57]:
keras.backend.clear_session()

# small model
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8

UNITS = 512
DROPOUT = 0.1


# average model
# NUM_LAYERS = 6
# D_MODEL = 512
# NUM_HEADS = 8
# UNITS = 2048
# DROPOUT = 0.1


model = transformer(
    vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    max_len=[max_len_en, max_len_ru])


optimizer = keras.optimizers.Adam(
    0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    return keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
checkpoint = keras.callbacks.ModelCheckpoint('model_ruen.weights.h5',
                                            monitor='val_loss',
                                            verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

In [58]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
# model.load_weights('model_ruen.weights.h5')

In [59]:
model.fit((X_en_train, X_ru_dec_train), X_ru_out_train,
             validation_data=((X_en_valid, X_ru_dec_valid), X_ru_out_valid),
             batch_size=200,
             epochs=3,
             callbacks=[checkpoint]
             )

Epoch 1/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - accuracy: 0.0880 - loss: 1.6289
Epoch 1: val_loss improved from inf to 0.98398, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 1655s 348ms/step - accuracy: 0.0880 - loss: 1.6289 - val_accuracy: 0.1465 - val_loss: 0.9840
Epoch 2/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - accuracy: 0.1471 - loss: 0.9739
Epoch 2: val_loss improved from 0.98398 to 0.88115, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 1663s 350ms/step - accuracy: 0.1471 - loss: 0.9739 - val_accuracy: 0.1587 - val_loss: 0.8811
Epoch 3/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - accuracy: 0.1583 - loss: 0.8766
Epoch 3: val_loss improved from 0.88115 to 0.84232, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 1665s 350ms/step - accuracy: 0.1583 - loss: 0.8766 - val_accuracy: 0.1639 - val_loss: 0.8423


In [60]:
@torch.no_grad()
def translate(text):
    input_ids = encode(text, tokenizer_en, target=False)

    input_ids = keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [input_ids], maxlen=max_len_en, padding='post',
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32)



    output_ids = [tokenizer_ru.token_to_id('[START]') ]

    pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [output_ids], maxlen=max_len_ru-1, padding='post',
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32))).cpu().numpy()


    while pred.argmax(2)[0][-1] not in [tokenizer_ru.token_to_id('[END]')]:
        if len(output_ids) >= max_len_ru:
            break
        # можно занизить скор тэга UNK чтобы он никогда не генерировался
        pred[:, :, tokenizer_ru.token_to_id('[UNK]')] = -100

        output_ids.append(pred.argmax(2)[0][-1])
        pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [output_ids], maxlen=max_len_ru-1, padding='post',
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32))).cpu().numpy()

    return tokenizer_ru.decode(output_ids[1:], )


In [61]:
translate("Transformer")

'трансформатора'

In [62]:
translate("can you translate this sentence?")

'можно translate thisate??'

In [66]:
translate("road works ahead - sure hope it does")

'дороти дорога работает.'

In [69]:
translate("i want to buy a bag")

'я хочу купить мешок.'

# BLEU

Реализация bleu есть в nltk

In [ ]:
# %pip install nltk

In [70]:
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']


BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True)
print(BLEUscore)

0.4548019047027907


In [71]:
text = open('opus.en-ru-test.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-test.ru', 'w')
f.write(text)
f.close()

In [72]:
en_sents_test = open('opus.en-ru-test.en').read().lower().splitlines()
ru_sents_test = open('opus.en-ru-test.ru').read().lower().splitlines()

In [73]:
#НЕ ТРОГАТЬ ЯЧЕЙКУ И ПЕРЕМЕННУЮ
translations = []

for i in range(len(en_sents_test)):
    translations.append(translate(en_sents_test[i]))

In [74]:
en_sents_test[-10:]

['while i was tracking his i.p. address for castle,',
 "you're mom's baby.",
 'i would be if your brothers had anything to do with it.',
 'the bosnia and herzegovina council of ministers has also recommended to the entity ministries of administration and local administration, cantonal and municipal ministries and services responsible for this area, to ensure an institutional framework, as well as human and financial resources, in order to solve this problem during 2006.',
 "you just need to hear it's okay.",
 'i simply went for a drive and had an accident that affected my memory.',
 "the reception's at 6 p.m., dinner at 8 p.m.",
 'so i picked him up.',
 'problems would arise only if accession by reios affected the rights and obligations of other parties, but he was confident that that would not be the case.',
 'the requested url /high_precision_die_cutting_machine_hydraulic_type.html was not found on this server.']

In [75]:
ru_sents_test[-10:]

['я отследила его ip-адрес по просьбе касла.',
 'ты мой сыночек.',
 'я мог бы стать им, если бы твои браться вмешались.',
 'были организованы практикумы и общественные слушания по следующим вопросам: профилактика и охрана здоровья и защита жертв насилия.',
 '- тед, ты азартен. - ты хочешь выиграть.',
 'я не собирался скрываться, сэр. я катался и попал в аварию, что и повлияло на мою память.',
 'прием в 6 часов, ужин в 8!',
 'я его снял.',
 'проблемы могут возникнуть только в том случае, если присоединение роэи будет затрагивать права и обязанности других сторон, однако он уве-рен, что этого не произойдет.',
 'the requested url /pet_bottles_recycling_a_level.html was not found on this server.']

In [76]:
translations[-10:]

['пока я отслеживал егочу, я.',
 'ты ребенок мамы.',
 'я буду делать что с с. если у твои',
 'в 2006 году в. в году в г. г. г. г. г. г. г. г. г. г г. г г г г н. центр по',
 'тебе просто нужно слышать.',
 'я просто пришла на и в авариююююююююююююююю.',
 'прием в,, вееее в часов вечера. вх..',
 'я его под ног.',
 'проблемы возникают только лишь, если не затро в условиях защиты прав и и других,,, не он был уверен. он уверен, не.',
 'the requested url / / / /_08 / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / /.']

In [77]:
bleus = []

for i, t in enumerate(translations):
    reference = tokenizer_ru.encode(t).tokens
    hypothesis = tokenizer_ru.encode(ru_sents_test[i]).tokens

    bleus.append(round(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  auto_reweigh=True), 3))

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [78]:
(sum(bleus)/len(bleus))*100

2.5727499999999996

In [89]:
indexed_list = list(enumerate(bleus))
sorted_list = sorted(indexed_list, key=lambda x: x[1], reverse=True)

top_15_indices = [index for index, _ in sorted_list[:15]]
top_15_values = [value for _, value in sorted_list[:15]]

for i, (index, value) in enumerate(zip(top_15_indices, top_15_values), 1):
    print(f"Top {i}: Index = {index}, Value = {value}")

Top 1: Index = 62, Value = 1.0
Top 2: Index = 138, Value = 1.0
Top 3: Index = 404, Value = 1.0
Top 4: Index = 702, Value = 1.0
Top 5: Index = 980, Value = 1.0
Top 6: Index = 1260, Value = 1.0
Top 7: Index = 1460, Value = 1.0
Top 8: Index = 1650, Value = 1.0
Top 9: Index = 1353, Value = 0.819
Top 10: Index = 1869, Value = 0.809
Top 11: Index = 358, Value = 0.779
Top 12: Index = 1359, Value = 0.76
Top 13: Index = 260, Value = 0.717
Top 14: Index = 1206, Value = 0.717
Top 15: Index = 1432, Value = 0.717


In [90]:
for index in top_15_indices:
    original_sentence = en_sents_test[index]
    translated_sentence = translations[index]
    print(f'{original_sentence} - {translated_sentence}')

12844 - 12844
siberian federal university - сибирский федеральный университет
s5000 - s5000
98799 - 98799
he was ordered to find a solution. - он приказал найти решение.
that's not the uterus. - это не матка.
jon: what is this? - что это?
320757 - 320757
- then tell me the truth - - - тогда скажи правду.
agenda item 109 (a) - пункт 109 ( a ) повестки повестки
our friend over in brussels? - наш друг в брюсселе?
4,907usd - 4, 907us
- yeah. - - - да...
i'm almost back. - я почти вернулся.
i don't know. - я не знаю.


Хорошо переводятся числа (хотя их не сложно переводить😇) и короткие фразы


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/13.pdf
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как ее применить к паре en->ru на данных из семинара. Сколько моделей понадобится? Сколько запусков обучения нужно будет сделать?

Ответ должен содержать как минимум 10 предложений.

## Ответ
backtranslation - распространенная техника аугментации данных при машинном переводе. Ее применяют, когда в налчиии нет достаточно большого параллельного корпуса и поэтмоу приходится его создавать самостоятельно. При этом для того, чтобы применить такую технику необходим большой корпус на языке-таргете. По сути, мы пользуемся тем, что имеем, чтобы синтетически дополнить параллеьный корпус. В процессе выполняются следующие шаги:  


1.   Ставим задачу перевода с языка Source на язык Target (в нашем случае S=en, T=ru)
2.   Если у нас маленький параллельный корпус (en-ru), то появляется повод применить технику backtranslation
3.   Находим достаточно объемный корпус на языке Target (на русском).
4.   Обучаем модель на имеющемся параллельном корпусе, но учим модель в обратную сторону, то есть переводитть с Target на Source (с русского на английский)
5.   Так как наша модель умеет переводить с Target на Source, мы можем с ее помощью перевести найденный в пункте 3 корпус на языке Target (то есть мы переведем наш русский корпус на английский). Таким образом мы сами сгенерируем параллельный корпус и сможем добавить его к тому небольшому, что у нас есть.
6.   Сгенерированные параллельные тексты можно добавить к training сету и обучить новую модель переводить с языка Source на язык Target (с англ на рус)

Таким образом, применяя технику backtranslation к паре en->ru потребуется одна модель для обучения ru->en и одна модель для обучения en->ru.



